1. 기본 세팅(아래 코드)
os -> file 탐색을 위해
re -> CT dicom 파일 뒤에 있는 인덱스 숫자 추출하기 위해
pydicom, numpy -> 당연히 있어야
collections, deque -> 점 이어주기를 위해 수행하는 BFS 를 위한 queue

In [74]:
import os
import re
import pydicom
import numpy as np
from collections import deque
basepath = 'D:\CodeMaster\w7_dataset'
CT_folder_path = basepath+'\P001_HN1_CT_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_Head.&.Neck..3.0..B31s_n150__00000'
RTst_folder_path = basepath + '\P001_HN1_RTst_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_re_n1__00000'
convertcolor = 65535/255

2. CT 정보 가져오기(아래코드) CT 정보를 가져오고, PixelSpacing과 SliceThickness, CT의 시작이미지나 끝 이미지로 부터 이미지의 기준 좌표(시작되는 원점으로 둘곳)를 가져옴. 회사마다 시작 이미지로해야할지 끝 이미지로 해야할지가 달라져서 세팅해주어야함.

In [75]:
#get CT info and number of slice
def GetCTinfo(CT_folder_path):
    filename_list = []
    fileidx = []
    for top, dir, file in os.walk(CT_folder_path):
        for filename in file:
            if filename[-3:] != 'dcm':
                continue
            filename_list.append(filename)
            idxnum = re.findall(r'\d+', filename)[-1]
            fileidx.append(int(idxnum))
        break
    ####이부분 instancenumber를 뽑아 정렬하는 과정 필요함####

    fileidx = np.array(fileidx)
    fileorder = np.argsort(fileidx)
    
    sorted_filename_list = []
    sorted_filename_list.append(filename_list[fileorder[0]])
    first_ct = pydicom.dcmread(CT_folder_path + '/'+sorted_filename_list[0])
    print(sorted_filename_list[0])

    return first_ct, len(fileidx)
    #좌표변환의 기준이 되는건 항상 '발쪽방향에서 첫번째 사진'
    #머리 끝쪽에서 하는 경우 뒤집어줘야함.
    #자동변환을 위해

first_ct, zsize = GetCTinfo(CT_folder_path)


2.16.840.1.114362.1.11745409.22133452282.537028968.978.4543.dcm


3. 2번에서 가져온 PixelSpacing과 ImagePositionPatient로 부터, scale 및 translation 할 값을 계산

In [76]:
def ComputeScaleValue(first_ct):
    #standard: SIEMENS
    scale_value = [float(first_ct.PixelSpacing[0]),
                   float(first_ct.PixelSpacing[1]),
                   float(first_ct.SliceThickness)]

    translation = [abs(first_ct.ImagePositionPatient[0])/first_ct.PixelSpacing[0], 
                   abs(first_ct.ImagePositionPatient[1])/first_ct.PixelSpacing[1], 
                   abs(first_ct.ImagePositionPatient[2])/first_ct.SliceThickness]

    if first_ct.Manufacturer == 'TOSHIBA': #TOSHIBA일 경우 좌표 보정(사진순서가 다름)
        translation[2] -= 1
    
    return scale_value, translation
scale_value, translation = ComputeScaleValue(first_ct)

4. RTst로부터 contour정보를 가져옴.

In [77]:
def GetRTstinfo(RTst_folder_path):
    for top, dir, file in os.walk(RTst_folder_path):
        for filename in file:
            if filename[-3:] != 'dcm':
                continue
            print(RTst_folder_path +'/'+filename)
            dcm = pydicom.dcmread(RTst_folder_path + '/' + filename, force=True)
            break
    all_ctr = dcm.ROIContourSequence #ROIContourSequence로 Contour 좌표 불러올 수 있음.
    return all_ctr
all_ctr = GetRTstinfo(RTst_folder_path)

D:\CodeMaster\w7_dataset\P001_HN1_RTst_2018-11-01_102652_RT^01.RT.Head.Neck.(Adult)_re_n1__00000/2.16.840.1.114362.1.11745409.22133452282.538218382.303.4908.dcm


5. 7번에 쓰일 함수: 1차원으로 정리되어있는 contour 좌표를, 3개로 끊어서 2차원 배열로 저장하고, 그것을 또 volumecoord로 저장.

In [78]:
def GetConvertedContourCoordinate(all_ctr, ctrs):
    ctr_volume_coord = []
    R, G, B = int(all_ctr[ctrs].ROIDisplayColor[0]),int(all_ctr[ctrs].ROIDisplayColor[1]),int(all_ctr[ctrs].ROIDisplayColor[2])
    for _slice in range(len(all_ctr[ctrs].ContourSequence)):
        ctr_coord_1dim = all_ctr[ctrs].ContourSequence[_slice].ContourData 
        coord_arr = np.zeros((1, 3))
        ###I will append...###
        for idx in range(0, len(ctr_coord_1dim), 3): 
            #1차원적 데이터인 ContourData를 3차원으로(데이터는 3의 배수이므로 다음과같이)
            ##################|해당축 기준좌표|/(해당 축 resolution) -> 넘파이에 넣을 좌표에 더할값####################
            coord_arr = np.append(coord_arr, 
            [[(round(ctr_coord_1dim[idx]/scale_value[0]+translation[0])), 
            (round(ctr_coord_1dim[idx+1]/scale_value[1]+translation[1])), 
            (round(ctr_coord_1dim[idx+2]/scale_value[2]+translation[2]))]],
            axis = 0) 
        coord_arr = np.delete(coord_arr, 0, axis = 0)
        ctr_volume_coord.append(coord_arr) 
    rgbset = (R, G, B)
    
    return ctr_volume_coord, rgbset

6. 7번에 쓰일 함수: 3개씩 담긴 배열로 정리된 좌표 정보를 바탕으로, 비어있는 voxel에 RGB정보를 채워줌.
이때 비어지고 끊어있는 선은 BFS를 활용하여 다음 점까지 std_distance를 업데이트하면서, this_distance가 std_distance보다 크면 넘어가고 작으면 queue에 넣는방식으로 탐색함.
조건을 만족하여 queue에 넣어질 pixel은 해당 contour의 RGB정보를 담아줌.

In [79]:
def ConnectContourMachine(voxelnp, ctr_volume_coord, rgbset):
    R, G, B = rgbset
    for idx, _slice in enumerate(ctr_volume_coord):
            for point_idx in range(len(_slice)):
                zi, yi, xi = _slice[point_idx][2], _slice[point_idx][1], _slice[point_idx][0]
                try:ziplus, yiplus, xiplus = _slice[point_idx+1][2], _slice[point_idx+1][1], _slice[point_idx+1][0]
                except:ziplus, yiplus, xiplus = _slice[0][2], _slice[0][1], _slice[0][0]

                ##################bfs searching for next point#################
                ########################connecting pixel#######################
                std_distance = ((xi-xiplus)**2 + (yi-yiplus)**2)**0.5
                if std_distance > 20:
                    continue
                queue = deque()
                dx = [1, -1, 0, 0]
                dy = [0, 0, 1, -1]
                voxelnp[int(zi)][int(yi)][int(xi)] = \
                    np.array([round(R*convertcolor), round(G*convertcolor), round(B*convertcolor)])
                queue.append((xi, yi))
                while queue:
                    x, y = queue.popleft()
                    for i in range(4):
                        nx = x + dx[i]
                        ny = y + dy[i]
                        this_distance = ((nx-xiplus)**2+(ny-yiplus)**2)**0.5
                        if std_distance <= this_distance:
                            continue
                        else:
                            std_distance = this_distance
                            if (nx, ny) == (xiplus, yiplus):
                                break
                            queue.append((nx, ny))
                            voxelnp[int(zi)][int(ny)][int(nx)] = \
                                np.array([round(R*convertcolor), round(G*convertcolor), round(B*convertcolor)])
    
    return voxelnp

7. voxelnp를 CT의 z축 size에 맞게 zeros배열로 선언해주고 위의 두 함수로 voxelnp를 채워나감

In [80]:
def MakeContourVoxel(all_ctr, zsize):
    voxelnp = np.zeros((zsize, 512, 512, 3))
    for ctrs in range(len(all_ctr)):
        ctr_volume_coord, rgbset = GetConvertedContourCoordinate(all_ctr, ctrs)
        voxelnp = ConnectContourMachine(voxelnp, ctr_volume_coord, rgbset)
    voxelnp = voxelnp.astype('uint16')
    return voxelnp

voxelnp = MakeContourVoxel(all_ctr, zsize)


8. 저장하기

In [81]:
np.save('D:\CodeMaster\w7voxels\p001rtst\p001voxel_ctr', voxelnp)